In [1]:
from model import *
from utils import *
from dataset import *

2025-04-14 12:42:39.492354: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-14 12:42:40.027791: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/supernova/anaconda3/envs/tensorflow/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/supernova/

In [ ]:
file_path = r'./TCN4Flare_agn_data/'
sub_path = 'ztf_agns/'
data_path = r'./TCN4Flare_agn_data/dataset/'

In [3]:
Flare_Dataset = FlareDataset(file_path, sub_path, data_path)

raw_flare_dataset, flare_params = Flare_Dataset.extarct_data_insert_flares('zg')

print('Flare dataset shape:', raw_flare_dataset.shape)
print('Flare params shape:', flare_params.shape)

Flare dataset shape: (12067, 4404, 3)
Flare params shape: (12067, 8)


In [4]:
# if may take a long time to run, please be patient.
fit_flare_dataset, flare_agn_params, fail_agns = Flare_Dataset.GP_fit_raw_dataset(band='zg', raw_dataset=raw_flare_dataset, agn_params=flare_params)
print(fit_flare_dataset.shape)
print(flare_agn_params.shape)
print(fail_agns.shape)

/home/supernova/anaconda3/envs/tensorflow/lib/python3.10/site-packages/celerite/terms.py:566: RuntimeWarning: overflow encountered in double_scalars
  return (w0*S0, w0*w0*S0/self.eps, w0, self.eps)
/home/supernova/anaconda3/envs/tensorflow/lib/python3.10/site-packages/scipy/optimize/_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0
/home/supernova/anaconda3/envs/tensorflow/lib/python3.10/site-packages/celerite/terms.py:564: RuntimeWarning: overflow encountered in exp
  w0 = np.sqrt(3.0) * np.exp(-log_rho)
/home/supernova/anaconda3/envs/tensorflow/lib/python3.10/site-packages/celerite/terms.py:566: RuntimeWarning: invalid value encountered in double_scalars
  return (w0*S0, w0*w0*S0/self.eps, w0, self.eps)


(12067, 2167, 3)
(12067, 10)
(0,)


In [5]:
# checked_idx = []

# # check if the flare flux is less than 100, if yes, delete the row
# for i in range(fit_flare_dataset.shape[0]):
#     flux = fit_flare_dataset[i,:,1]
#     if flux.max() < 100:
#         checked_idx.append(i)

# fit_flare_dataset = np.delete(fit_flare_dataset, checked_idx, axis=0)
# flare_agn_params = np.delete(flare_agn_params, checked_idx, axis=0)

# Directly load checked flare dataset and agn parameters, Num = 705
flare_dataset_all = np.load(r'./data/fit_checked_flares.npz')
flare_dataset = flare_dataset_all['data']
flare_agn_params = flare_dataset_all['parms']   # parms contains: objid，t_start，t_end，t_duration，mag_peak，shape。(705, 1, 6)

print(flare_dataset.shape)
print(flare_agn_params.shape)

(705, 2167, 3)
(705, 6)


In [6]:
noflare_Dataset = dataset(file_path, sub_path, data_path)
raw_noflare_dataset, noflare_agn_params, empty_files = noflare_Dataset.extract_data(band='zg')

print(raw_noflare_dataset.shape)
print(noflare_agn_params.shape)
print(len(empty_files))

(17037, 4404, 3)
(17037, 3)
2963


In [ ]:
fit_noflare_dataset, noflare_agn_params, fail_agns = noflare_Dataset.GP_fit_raw_dataset('zg', raw_noflare_dataset, noflare_agn_params)

print(fit_noflare_dataset.shape)
print(noflare_agn_params.shape)

In [8]:
# idx = []    # check noflare dataset, find the index of the data with flare
# noflare_dataset = np.delete(fit_noflare_dataset, idx, axis=0)
# noflare_agn_params = np.delete(noflare_agn_params, idx, axis=0)

# Directly load checked noflare dataset
noflare_dataset = np.load(r'./data/fit_checked_noflares.npy')

print(noflare_dataset.shape)

(19710, 2167, 3)


In [10]:
train_dataset , train_labels = build_dataset(flare_dataset, noflare_dataset, save_path=r'./data/', save_name='train_dataset', ratio=0.1, num = 5000)

print(train_dataset.shape, train_labels.shape)

(5000, 2167, 3) (5000,)
